In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gc
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from typing import overload, Union, Literal

from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torch.hub import load_state_dict_from_url
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR
from tqdm import tqdm

from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

from torch.autograd import Variable

In [ ]:
N_ATTACKS = 3
N_SUBNET_LABELS = N_ATTACKS + 1 # add label for non-adversarial

In [ ]:
def _weights_init(m):
    classname = m.__class__.__name__
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x, return_interm_layer=None):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        if return_interm_layer == 1:
            return out
        out = self.layer2(out)
        if return_interm_layer == 2:
            return out
        out = self.layer3(out)
        if return_interm_layer == 3:
            return out
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        
        return out


def resnet20():
    return ResNet(BasicBlock, [3, 3, 3])


def resnet32():
    return ResNet(BasicBlock, [5, 5, 5])


def resnet44():
    return ResNet(BasicBlock, [7, 7, 7])


def resnet56():
    return ResNet(BasicBlock, [9, 9, 9])


def resnet110():
    return ResNet(BasicBlock, [18, 18, 18])


def resnet1202():
    return ResNet(BasicBlock, [200, 200, 200])

In [ ]:
class MultiClassSubNet(nn.Module):
    def __init__(self, in_channels):
        super(MultiClassSubNet, self).__init__()
        conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=2, padding=0, bias=False)
        bn1 = nn.BatchNorm2d(64)
        conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=0, bias=False)
        bn2 = nn.BatchNorm2d(128)
        conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=0, bias=False)
        bn3 = nn.BatchNorm2d(256)
        relu = nn.ReLU(inplace=True)
        flatten = nn.Flatten()
        linear = nn.Linear(256, N_SUBNET_LABELS)

        self.layers = nn.Sequential(
            conv1,
            bn1,
            relu,
            conv2,
            bn2,
            relu,
            conv3,
            bn3,
            relu,
            flatten,
            linear,
        )

        self.layers.apply(self.init_param)

    def forward(self, x, return_embedding=False):
        embedding = None
        for itr, layer in enumerate(self.layers):
          x = layer(x)
          if return_embedding and itr == len(self.layers)-1:
            embedding = x

        return x, embedding

    def init_param(self, param):
        if type(param) in [nn.Linear, nn.Conv2d]:
            nn.init.kaiming_uniform_(param.weight)

In [ ]:
class MLP(nn.Module):

    def __init__(self, size):
        super(MLP, self).__init__()
                
        self.model = nn.Sequential(
            nn.Linear(size[0], size[1]), 
            nn.ReLU(),
            nn.Linear(size[1], size[2]), 
            nn.ReLU(),
            nn.Linear(size[2], size[3]), 
            nn.ReLU(),
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
class End2EndSubNetsWithMLP(nn.Module):

    def __init__(self, resnet, subnets: list, interm_layers: list, mlp):
        super(ToolChainClassifier, self).__init__()

        self.subnets = subnets
        self.mlp = mlp

    def forward(self, subnet_inputs):
        subnet_outputs = []
        for subnet_idx in range(len(self.subnets)):
          curr_subnet_output = self.subnets[subnet_idx](subnet_inputs[subnet_idx])
          subnet_outputs.append(curr_subnet_output)

        mlp_input = torch.cat(subnet_outputs, axis=0)
        mlp_output = self.mlp(mlp_input)

        return mlp_output

### End-to-End Training

In [ ]:
def train_multi_subnet_end2end(
    resnet_model,
    interm_layers: list,
    subnet_models: list,
    mlp_model=None,
    joint_optimizer=None,
    joint__criterion=None,
    joint_scheduler=None,
    unattacked_train_data=None,
    unattacked_test_data=None,
    attacked_train_data_list=None,
    attacked_test_data_list=None,
    device='cpu',
    epochs=100,
    batch_size=64,
    finetune=False,
):
    for subnet_model in subnet_models:
      subnet_model.train()
    resnet_model.eval()
    batches = []

    for epoch in range(epochs):

      avg_loss = 0.0
      for batch_itr in tqdm(range(0, len(unattacked_train_data), batch_size)):
          
          all_attacked_input, all_attacked_labels = [], []
          for label_id, attacked_train_data in enumerate(attacked_train_data_list):
            attacked_input = attacked_train_data[batch_itr:batch_itr+batch_size]
            all_attacked_input.extend(attacked_input)

            attacked_labels = torch.ones((attacked_input.shape[0], label_id+1), dtype=torch.float32)
            all_attacked_labels.extend(attacked_labels)

          all_attacked_input = torch.cat(all_attacked_input, axis=0)
          all_attacked_labels = torch.cat(all_attacked_labels, axis=0)

          unattacked_input = unattacked_train_data[batch_itr:batch_itr+batch_size]
          unattacked_labels = torch.zeros((unattacked_input.shape[0], 1), dtype=torch.float32)

          input = torch.cat((unattacked_input, all_attacked_input), axis=0)
          labels = torch.cat((unattacked_labels, all_attacked_labels), axis=0)

          input, labels = input.to(device), labels.to(device)

          for subnet_idx in range(len(subnet_models)):
            resnet_output = resnet_model(
                input.float(), return_interm_layer=interm_layers[subnet_idx]
            )
            subnet_output = subnet_models[subnet_idx](resnet_output)

          input = resnet_model(input.float(), return_interm_layer=interm_layer)
          output = subnet_model(input)

          loss = subnet_criterion(output, labels)
          loss.backward()
          avg_loss += loss.item()

          subnet_optimizer.step()

          del input
          del labels
          del loss
          torch.cuda.empty_cache()

      val_loss, val_acc = test_multi_subnet_end2end(
          resnet_model,
          interm_layer,
          subnet_model,
          subnet_criterion,
          unattacked_test_data,
          attacked_test_data_list,
          device,
      )

      print('Val Loss: {:.4f} | Val Accuracy: {:.4f}'.format(val_loss, val_acc))
      torch.save({
          'epoch': epoch,
          'model_state_dict': resnet_model.state_dict(),
          'optimizer_state_dict': subnet_optimizer.state_dict(),
      }, './' + str(epoch) + 'model.pt')

      # subnet_scheduler.step(val_loss)

def test_multi_subnet_end2end(
    resnet_model,
    interm_layers: list,
    subnet_models: list,
    subnet_criterions: list,
    mlp_model=None,
    mlp_criterion=None,
    unattacked_test_data=None,
    attacked_test_data_list=None,
    device='cpu',
    batch_size=64,
    test_mode: Literal["majority_vote", "mlp"] = "mlp",
):
    resnet_model.eval()
    subnet_model.eval()
    test_loss = []
    accuracies = []

    for batch_itr in tqdm(range(0, len(unattacked_test_data), batch_size)):
        all_attacked_input, all_attacked_labels = [], []
        for label_id, attacked_test_data in enumerate(attacked_test_data_list):
          attacked_input = attacked_test_data[batch_itr:batch_itr+batch_size]
          all_attacked_input.extend(attacked_input)

          attacked_labels = torch.ones((attacked_input.shape[0], label_id+1), dtype=torch.float32)
          all_attacked_labels.extend(attacked_labels)

        all_attacked_input = torch.cat(all_attacked_input, axis=0)
        all_attacked_labels = torch.cat(all_attacked_labels, axis=0)

        unattacked_input = unattacked_test_data[batch_itr:batch_itr+batch_size]
        unattacked_labels = torch.zeros((unattacked_input.shape[0], 1), dtype=torch.float32)

        input = torch.cat((unattacked_input, all_attacked_input), axis=0)
        labels = torch.cat((unattacked_labels, all_attacked_labels), axis=0)

        input, labels = input.to(device), labels.to(device)

        with torch.no_grad():
            input = resnet_model(input.float(), return_interm_layer=interm_layer)
            output = subnet_model(input)

        pred_labels = torch.argmax(output).float()
        loss = criterion(output, labels)

        accuracy = accuracy_score(pred_labels.flatten().cpu(), labels.flatten().cpu())
        accuracies.append(accuracy)
        test_loss.extend([loss.item()]*input.size()[0])
        
        del input
        del labels
        del loss
        torch.cuda.empty_cache()

    subnet_model.train()

    return np.mean(test_loss), np.mean(accuracies)

### Individual Training

In [ ]:
def train_subnet(
    resnet_model,
    interm_layer,
    subnet_model,
    subnet_optimizer,
    subnet_criterion,
    subnet_scheduler,
    unattacked_train_data,
    unattacked_test_data,
    attacked_train_data_list,
    attacked_test_data_list,
    device,
    epochs=100,
    batch_size=64,
):
    subnet_model.train()
    resnet_model.eval()
    batches = []

    for epoch in range(epochs):

      avg_loss = 0.0
      for batch_itr in tqdm(range(0, len(unattacked_train_data), batch_size)):
          
          all_attacked_input, all_attacked_labels = [], []
          for label_id, attacked_train_data in enumerate(attacked_train_data_list):
            attacked_input = attacked_train_data[batch_itr:batch_itr+batch_size]
            all_attacked_input.extend(attacked_input)

            attacked_labels = torch.ones((attacked_input.shape[0], label_id+1), dtype=torch.float32)
            all_attacked_labels.extend(attacked_labels)

          all_attacked_input = torch.cat(all_attacked_input, axis=0)
          all_attacked_labels = torch.cat(all_attacked_labels, axis=0)

          unattacked_input = unattacked_train_data[batch_itr:batch_itr+batch_size]
          unattacked_labels = torch.zeros((unattacked_input.shape[0], 1), dtype=torch.float32)

          input = torch.cat((unattacked_input, all_attacked_input), axis=0)
          labels = torch.cat((unattacked_labels, all_attacked_labels), axis=0)

          input, labels = input.to(device), labels.to(device)

          input = resnet_model(input.float(), return_interm_layer=interm_layer)
          output = subnet_model(input)

          loss = subnet_criterion(output, labels)
          loss.backward()
          avg_loss += loss.item()

          subnet_optimizer.step()

          del input
          del labels
          del loss
          torch.cuda.empty_cache()

      val_loss, val_acc = test_subnet(
          resnet_model,
          interm_layer,
          subnet_model,
          subnet_criterion,
          unattacked_test_data,
          attacked_test_data_list,
          device,
      )

      print('Val Loss: {:.4f} | Val Accuracy: {:.4f}'.format(val_loss, val_acc))
      torch.save({
          'epoch': epoch,
          'model_state_dict': resnet_model.state_dict(),
          'optimizer_state_dict': subnet_optimizer.state_dict(),
      }, './' + str(epoch) + 'model.pt')

      # subnet_scheduler.step(val_loss)

def test_subnet(
    resnet_model,
    interm_layer,
    subnet_model,
    criterion,
    unattacked_test_data,
    attacked_test_data_list,
    device,
    batch_size=64,
):
    resnet_model.eval()
    subnet_model.eval()
    test_loss = []
    accuracies = []

    for batch_itr in tqdm(range(0, len(unattacked_test_data), batch_size)):
        all_attacked_input, all_attacked_labels = [], []
        for label_id, attacked_test_data in enumerate(attacked_test_data_list):
          attacked_input = attacked_test_data[batch_itr:batch_itr+batch_size]
          all_attacked_input.extend(attacked_input)

          attacked_labels = torch.ones((attacked_input.shape[0], label_id+1), dtype=torch.float32)
          all_attacked_labels.extend(attacked_labels)

        all_attacked_input = torch.cat(all_attacked_input, axis=0)
        all_attacked_labels = torch.cat(all_attacked_labels, axis=0)

        unattacked_input = unattacked_test_data[batch_itr:batch_itr+batch_size]
        unattacked_labels = torch.zeros((unattacked_input.shape[0], 1), dtype=torch.float32)

        input = torch.cat((unattacked_input, all_attacked_input), axis=0)
        labels = torch.cat((unattacked_labels, all_attacked_labels), axis=0)

        input, labels = input.to(device), labels.to(device)

        with torch.no_grad():
            input = resnet_model(input.float(), return_interm_layer=interm_layer)
            output = subnet_model(input)

        pred_labels = torch.argmax(output).float()
        loss = criterion(output, labels)

        accuracy = accuracy_score(pred_labels.flatten().cpu(), labels.flatten().cpu())
        accuracies.append(accuracy)
        test_loss.extend([loss.item()]*input.size()[0])
        
        del input
        del labels
        del loss
        torch.cuda.empty_cache()

    subnet_model.train()

    return np.mean(test_loss), np.mean(accuracies)

In [ ]:
def Normalize0to1(AA):
    AA[:, :, :, 0] -= AA[:, :, :, 0].min().item()
    AA[:, :, :, 0] /= AA[:, :, :, 0].max().item()

    AA[:, :, :, 1] -= AA[:, :, :, 1].min().item()
    AA[:, :, :, 1] /= AA[:, :, :, 1].max().item()

    AA[:, :, :, 2] -= AA[:, :, :, 2].min().item()
    AA[:, :, :, 2] /= AA[:, :, :, 2].max().item()

    return AA / 225.

In [ ]:
batch_size = 64

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

unattacked_data_path = "/content/drive/MyDrive/11785/Project/unattack_test.npy"
unattacked_data = np.load(unattacked_data_path, allow_pickle=True).astype(float)
unattacked_data = torch.from_numpy(unattacked_data.transpose(0, 3, 1, 2))
unattacked_data = Normalize0to1(unattacked_data)
unattacked_data = normalize(unattacked_data)
unattacked_train_data = unattacked_data[:9000]
unattacked_test_data = unattacked_data[9000:]

attacked_data_path = "/content/drive/MyDrive/11785/Project/attack_new.npy"
attacked_data = np.load(attacked_data_path, allow_pickle=True).astype(float)
attacked_data = torch.from_numpy(attacked_data.transpose(0, 3, 1, 2))
attacked_data = Normalize0to1(attacked_data)
attacked_data = normalize(attacked_data)
attacked_train_data = attacked_data[:9000]
attacked_test_data = attacked_data[9000:]

# TODO: hack for testing, change to list of actual attacks
attacked_train_data_list = [attacked_train_data] * 3
attacked_test_data_list = [attacked_test_data] * 3

In [ ]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

resnet_model = resnet32()
resnet_model.to(device)
checkpoint = torch.load("/content/drive/MyDrive/11785/Project/resnet32.th")
mod_checkpoint = {k.replace("module.", ""): v for k, v in checkpoint['state_dict'].items()}
resnet_model.load_state_dict(mod_checkpoint)
resnet_optimizer = torch.optim.SGD(resnet_model.parameters(), lr=0.1, weight_decay=5e-5, momentum=0.9)
resnet_criterion = nn.CrossEntropyLoss()
resnet_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(resnet_optimizer, T_0=10, T_mult=2, eta_min=0.01, last_epoch=-1)

# output size at diff intermediate layers of resnet
interm_layer2dim = {1: 16, 2: 32, 3: 64}
interm_layer = 2

subnet_model = MultiClassSubNet(interm_layer2dim[interm_layer])
subnet_model.to(device)
subnet_optimizer = torch.optim.Adam(subnet_model.parameters(), lr=0.0001, betas=(0.99, 0.999))
subnet_criterion = nn.CrossEntropyLoss()
subnet_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(subnet_optimizer, T_0=10, T_mult=2, eta_min=0.01, last_epoch=-1)

In [ ]:
train_subnet(
  resnet_model,
  interm_layer,
  subnet_model,
  subnet_optimizer,
  subnet_criterion,
  subnet_scheduler,
  unattacked_train_data,
  unattacked_test_data,
  attacked_train_data_list,
  attacked_test_data_list,
  device,
  epochs=50,
)